In [1]:
import plotly.offline as pyo
import plotly.graph_objs as go
import pandas as pd

# Data Preparation

In [2]:
df_scores = pd.read_pickle('200608-150334-df_scores.pkl')

In [3]:
df_scores = df_scores.fillna(0)

In [4]:
df_clean = pd.read_pickle('200604-192804-df_clean.pkl')

In [5]:
df_clean['dc_score'] = (df_clean['d_score'] + df_clean['c_score']) / 2

In [6]:
df_count = df_clean[['crypto','bitcoin','ethereum']]
df_count = df_count.resample('D').sum()
df_count = df_count.sort_index(ascending=False)

In [7]:
df_scores = df_scores.merge(df_count, on='publishedAt')

# Daily Sentiment Plot

In [8]:
def daily_line_plot(df):
    trace1 = go.Scatter(x=df.index,
                    y=df['CRY_score'],
                    name = 'Crypto',
                    #fill='tozeroy',
                    mode = 'lines+markers',
                    marker=dict(color='#136dad'))
                    #visible='legendonly')

    trace2 = go.Scatter(x=df.index,
                    y=df['BTC_score'],
                    name = 'Bitcoin',
                    #fill='tozeroy',
                    mode = 'lines+markers',
                    marker=dict(color='#d4d12c'))

    trace3 = go.Scatter(x=df.index,
                        y=df['ETH_score'],
                        name = 'Ethereum',
                        #fill='tozeroy',
                        mode = 'lines+markers',
                        marker=dict(color='#4b5b75'))

    data = [trace1, trace2, trace3]

    layout = go.Layout(title='Daily Average Sentiment Score')

    fig = go.Figure(data=data, layout=layout)
    
    fig.add_shape(type="rect", 
                  x0=df.index[0], 
                  x1=df.index[-1], 
                  y0= 0, 
                  y1= -1, 
                  line=dict(width=0),
                  fillcolor="rgba(255, 0, 0, 0.05)")
    
    fig.add_shape(type="rect", 
                  x0=df.index[0], 
                  x1=df.index[-1], 
                  y0= 0, 
                  y1= 1, 
                  line=dict(width=0),
                  fillcolor="rgba(0, 255, 0, 0.05)")
    
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7,
                         label="1w",
                         step="day",
                         stepmode="backward"),
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    
    initial_range = [df.index[-7], df.index[-1]]

    fig['layout']['xaxis'].update(range=initial_range, autorange=False)
    fig['layout']['yaxis'].update(range=[-1, 1], autorange=False)

    fig.update_layout(xaxis_tickangle=-45)

    pyo.plot(fig, filename='avg_score.html')

In [9]:
#daily_line_plot(df_scores)

# Daily Count Plot

In [10]:
def daily_bar_plot(df):
    df = df.sort_index(ascending=True)
    
    trace1 = go.Bar(x=df.index,
                    y=df['crypto'],
                    name = 'Crypto',
                    marker=dict(color='#136dad'),
                    #visible='legendonly',
                    text=df['crypto'],
                    textposition='auto')

    trace2 = go.Bar(x=df.index,
                    y=df['bitcoin'],
                    name = 'Bitcoin',
                    marker=dict(color='#d4d12c'),
                    text=df['bitcoin'],
                    textposition='auto')

    trace3 = go.Bar(x=df.index,
                    y=df['ethereum'],
                    name = 'Ethereum',
                    marker=dict(color='#4b5b75'),
                    text=df['ethereum'],
                    textposition='auto')

    data = [trace1, trace2, trace3]

    layout = go.Layout(title='Daily Articles Count')

    fig = go.Figure(data=data, layout=layout)

    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=8,
                         label="1w",
                         step="day",
                         stepmode="backward"),
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"))
    
    initial_range = [df.index[-8], (df.index[-1]+pd.DateOffset(1))]
    fig['layout']['xaxis'].update(range=initial_range)
    
    fig.update_layout(xaxis_tickangle=-45)

    pyo.plot(fig, filename='count.html')

In [11]:
#daily_bar_plot(df_scores)

# Scatter Plot

In [12]:
def scatter_plot(df):
    data = [go.Scatter(x=df['t_score'],
                       y=df['dc_score'],
                       mode='markers')]
    layout = go.Layout(title='Scatter Plot')
    fig = go.Figure(data=data, layout=layout)
    pyo.plot(fig, filename='sentiment.html')

In [13]:
#scatter_plot(df_clean)

# Dashboard

In [14]:
import dash
import dash_core_components as dcc
import dash_html_components as html

In [1]:
app = dash.Dash()

NameError: name 'dash' is not defined

In [50]:
df = df_scores

In [17]:
colors = {
    'crypto': '#136dad',
    'bitcoin': '#d4d12c',
    'ethereum': '#4b5b75'}

app.layout = html.Div([
    html.H1('CryptoNews',
            style={'textAlign': 'center'}),
    
    html.Div('An API based on Flask. It collects and displays the latest news about cryptocurrency from various news sources.',
             style={'textAlign': 'center'}),

    dcc.Graph(id='count-bars',
        figure={'data':[ 
            go.Bar(x=df.index,
                    y=df['crypto'],
                    name = 'Crypto',
                    marker=dict(color='#136dad'),
                    text=df['crypto'],
                    textposition='auto'),
            go.Bar(x=df.index,
                    y=df['bitcoin'],
                    name = 'Bitcoin',
                    marker=dict(color='#d4d12c'),
                    text=df['bitcoin'],
                    textposition='auto'),
            go.Bar(x=df.index,
                    y=df['ethereum'],
                    name = 'Ethereum',
                    marker=dict(color='#4b5b75'),
                    text=df['ethereum'],
                    textposition='auto')],
                'layout': go.Layout(
                title = 'Daily Noise Meter',
                yaxis = {'title': 'Number of mentions in headline'},
                xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=8,
                             label="1w",
                             step="day",
                             stepmode="backward"),
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"))}),
    
    dcc.Graph(id='score-lines',
        figure = {'data':[
            go.Scatter(
                    x=df.index,
                    y=df['CRY_score'],
                    name = 'Crypto',
                    fill='tozeroy',
                    mode = 'lines+markers',
                    marker=dict(color='#136dad')),
            go.Scatter(
                    x=df.index,
                    y=df['BTC_score'],
                    name = 'Bitcoin',
                    fill='tozeroy',
                    mode = 'lines+markers',
                    marker=dict(color='#d4d12c')),
            go.Scatter(
                    x=df.index,
                    y=df['ETH_score'],
                    name = 'Ethereum',
                    fill='tozeroy',
                    mode = 'lines+markers',
                    marker=dict(color='#4b5b75'))],
                'layout': {'title': 'Daily Average Sentiment Score'}})
])

In [18]:
#if __name__ == '__main__':
#    app.run_server()

In [19]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

In [61]:
colors = {
    'crypto': '#136dad',
    'bitcoin': '#d4d12c',
    'ethereum': '#4b5b75'}

app.layout = html.Div([
    html.H1('CryptoNews',
            style={'textAlign': 'center'}),
    
    html.Div('An API based on Flask. It collects and displays the latest news about cryptocurrency from various news sources.',
             style={'textAlign': 'center'}),
  
    dcc.RangeSlider(
        id='date-range-slider',
        #marks=[{i: v} for i,v in enumerate(df.index)],
        value=[df.index[-1], df.index[0]]
    ),
    
    dcc.Graph(id='count-bars')
    ])
    
@app.callback(Output('count-bars', 'figure'),
          [Input('date-range-slider', 'value')])
def update_figure(value):
    return {'data': [go.Scatter(x=df.index,
                y=df['CRY_score'],
                name = 'Crypto',
                fill='tozeroy',
                mode = 'lines+markers',
                marker=dict(color='#136dad'))],
            'layout': go.Layout(
                xaxis=dict(
                    rangeselector=dict(
                        buttons=list([
                            dict(count=7,
                                 label="1w",
                                 step="day",
                                 stepmode="backward"),
                            dict(count=1,
                                 label="1m",
                                 step="month",
                                 stepmode="backward"),
                            dict(step="all")
                        ])
                    ),
                    autorange=False,
                    type="date",
                    range=value
                ))}
            

In [65]:
#if __name__ == '__main__':
#    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [35]:
df.head(1)

,CRY_score,BTC_score,ETH_score,crypto,bitcoin,ethereum
publishedAt,,,,,,
2020-06-03,0.128736,0.042248,0.077611,17.0,45.0,9.0


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

app.layout = html.Div([
    html.H1('CryptoNews',
            style={'textAlign': 'center'}),
    
    html.Div('An API based on Flask. It collects and displays the latest news about cryptocurrency from various news sources.',
             style={'textAlign': 'center'}),
    
    dcc.Dropdown(id='coin-picker',
                options=[
                    {'label': 'Crypto', 'value': 'CRY_score'},
                    {'label': 'Bitcoin', 'value': 'BTC_score'},
                    {'label': 'Ethereum', 'value': 'ETH_score'}],
                value='BTC_score'),
                #multi=True),
    
    dcc.Graph(id='graph',
        figure={'data':[go.Scatter(
            x=df.index,
            y=df['BTC_score'],
            #name = 'Bitcoin',
            #fill='tozeroy',
            mode = 'lines+markers',
            marker=dict(color='#d4d12c'))]}),
                       
    html.Div(id='my-div')
])

@app.callback(Output('my-div', 'children'),
              [Input('coin-picker', 'value')])
def update_graph(coin_names):
    df = df[coin_names]
    traces = []
    for name in coin_names:
        traces.append(go.Scatter(
            x=df.index,
            y=df[coin_name],
            #name = 'Bitcoin',
            #fill='tozeroy',
            mode = 'lines+markers',
            marker=dict(color='#d4d12c')))
    return 'You have selected "{}"'.format(value)
#{'data': traces,'layout': go.Layout(title='Daily Average Sentiment Score')}

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2020 02:04:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 02:04:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 02:04:18] "GET /_dash-dependencies HTTP/1.1" 200 -


In [54]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2020 01:38:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 01:38:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 01:38:38] "GET /_dash-dependencies HTTP/1.1" 200 -


In [44]:
data = [go.Scatter(x=df.index,
            y=df['BTC_score'],
            #name = 'Bitcoin',
            #fill='tozeroy',
            mode = 'lines+markers',
            marker=dict(color='#d4d12c'))]
    
layout = go.Layout(title='Daily Average Sentiment Score')

fig = go.Figure(data=data, layout=layout)
    
pyo.plot(fig, filename='dash_test.html')

'dash_test.html'